In [4]:
#%matplotlib qt
from __future__ import print_function

#import rospy
#import rosbag
import itertools
from  tqdm import tqdm
#from geometry_msgs.msg import PoseWithCovarianceStamped
#import tf
import numpy as np

import matplotlib.pyplot as plt
import os
import time

%load_ext autoreload
%autoreload 2

In [5]:
import GPy

#import wifi-localization

from Localization.sensormodel.gp import GP
#from Localization.sensormodel.hybrid import hGP #これはcommentでも良い
#Localizetion.sensormodel.gp→みやぐすく先生のgitから！

ModuleNotFoundError: No module named 'Localization'

In [7]:
# instead of rss.classes import ProcessedData
class ProcessedData():
    def __init__(self,X,Z,mac_address_dict):
        self.data = {'X':X, 'Y':Z, 'Var':Z}
        self.nm = X.shape[0]
        self.all_mac_dict = mac_address_dict

In [8]:
save_model = False

root_folder = '/mnt/matrix/rosbag/processed_data/nict'
map_name = 'udai-01'


print('\n####################\n  map: {:s} \n####################'.format(map_name)); time.sleep(0.2)
save_filename = '{0:s}/{1:s}/invisible-maps/{1:s}_WifiData.npz'.format(root_folder,map_name)

print('Load data')
data = np.load(save_filename,allow_pickle=True)
XY = data['XY']
Wifi = data['Wifi']
macdict = data['macdict'].item()

print('Wifi  : '  ,data['Wifi'].shape,
      '\nXY    : ',data['XY'].shape)

print('Create model')
traindata = ProcessedData(XY,Wifi,macdict)

# Train model
print('Training GP')
time.sleep(0.2)

kernel = GPy.kern.RBF(2)#+GPy.kern.RBF(2,lengthscale=5)
#kernel = GPy.kern.Matern32(2)

t0 = time.time()
model = GP(traindata.data,all_mac_dict=macdict)
model.optimize(kernel=kernel)
print(time.time()-t0)

print(model)

if save_model:
    save_filename = '{0:s}/{1:s}/invisible-maps/{1:s}_WifiModel.npz'.format(root_folder,map_name)
    print('Saving model',save_filename)
    np.savez_compressed(save_filename,param_array=model.gp.param_array)


####################
  map: udai-01 
####################
Load data
Wifi  :  (2288, 270) 
XY    :  (2288, 2)
Create model
Training GP


NameError: name 'GP' is not defined

# Create Map from trained model

In [9]:
# Wifi Map
res = 1.0

print('Creating Wifi map, res {:.2f}'.format(res))
time.sleep(0.2)

xmin,xmax = np.min(data['XY'][:,0]), np.max(data['XY'][:,0])
ymin,ymax = np.min(data['XY'][:,1]), np.max(data['XY'][:,1])

x = np.arange(np.floor(xmin-20),np.ceil(xmax+20),res)
y = np.arange(np.floor(ymin-20),np.ceil(ymax+20),res)
print(x.shape, y.shape)
xy = np.meshgrid(x,y)
XYtest = np.concatenate([xy[0].reshape(-1,1),xy[1].reshape(-1,1)],axis=1).astype(np.float32)

batch_eval = 5000
iterations = int(np.ceil(XYtest.shape[0]/batch_eval))

Predictions_mean = np.zeros((XYtest.shape[0],Wifi.shape[1]))
Predictions_var = np.zeros((XYtest.shape[0],1))

# Make predictions
for i in tqdm(range(iterations+1)):
    try:
        xy = XYtest[i*batch_eval:(i+1)*batch_eval]
        #print('xy in mem')
        predictions_mean,predictions_variance = model.predict(xy)
        #print('predictions')
        Predictions_mean[i*batch_eval:(i+1)*batch_eval,:] = predictions_mean
        Predictions_var[i*batch_eval:(i+1)*batch_eval,:]  = predictions_variance  
        #print('copied to cpu')
    except:
        xy = XYtest[i*batch_eval:-1]
        predictions_mean,predictions_variance = model.predict(xy)
        Predictions_mean[i*batch_eval:-1,:] = predictions_mean
        Predictions_var[i*batch_eval:-1]    = predictions_variance


# Save data
if save_model: 
    save_filename = '{0:s}/{1:s}/invisible-maps/{1:s}_WifiMap_res_{2:03d}.npz'.format(root_folder,
                                                                            map_name,int(res*100))
    np.savez_compressed(save_filename,Mean=Predictions_mean,Var=Predictions_var,macdict=macdict)

Creating Wifi map, res 1.00
(231,) (154,)


  0%|                                                     | 0/9 [00:00<?, ?it/s]


NameError: name 'model' is not defined